In [47]:
import sqlite3
import pandas as pd
  
# Connect to SQLite database
conn = sqlite3.connect(r'./data/vivino.db')
# Create a cursor object
cursor = conn.cursor()

In [48]:
query1 = """SELECT
    country_name,
    natural_wines_count,
    non_natural_wines_count,
    natural_acidic_wines_count,
    natural_sweet_wines_count,
    natural_tannic_wines_count,
    non_natural_acidic_wines_count,
    non_natural_sweet_wines_count,
    non_natural_tannic_wines_count
FROM (
    SELECT
        c.name AS country_name,
        SUM(CASE WHEN w.is_natural = 1 THEN 1 ELSE 0 END) AS natural_wines_count,
        SUM(CASE WHEN w.is_natural = 0 THEN 1 ELSE 0 END) AS non_natural_wines_count,
        SUM(CASE WHEN w.is_natural = 1 AND w.acidity > w.sweetness AND w.acidity > w.tannin THEN 1 ELSE 0 END) AS natural_acidic_wines_count,
        SUM(CASE WHEN w.is_natural = 1 AND w.sweetness > w.acidity AND w.sweetness > w.tannin THEN 1 ELSE 0 END) AS natural_sweet_wines_count,
        SUM(CASE WHEN w.is_natural = 1 AND w.tannin > w.acidity AND w.tannin > w.sweetness THEN 1 ELSE 0 END) AS natural_tannic_wines_count,
        SUM(CASE WHEN w.is_natural = 0 AND w.acidity > w.sweetness AND w.acidity > w.tannin THEN 1 ELSE 0 END) AS non_natural_acidic_wines_count,
        SUM(CASE WHEN w.is_natural = 0 AND w.sweetness > w.acidity AND w.sweetness > w.tannin THEN 1 ELSE 0 END) AS non_natural_sweet_wines_count,
        SUM(CASE WHEN w.is_natural = 0 AND w.tannin > w.acidity AND w.tannin > w.sweetness THEN 1 ELSE 0 END) AS non_natural_tannic_wines_count
    FROM
        countries c
    LEFT JOIN
        regions r ON c.code = r.country_code
    LEFT JOIN
        wines w ON r.id = w.region_id
    GROUP BY
        c.name
    ORDER BY non_natural_wines_count DESC
) AS subquery;"""

cursor.execute(query1)
results = cursor.fetchall()

# Print column headers
print("{:<20} {:<20} {:<20} {:<20} {:<20} {:<20} {:<20} {:<20} {:<20}".format(
    "Country", "Natural Wines", "Non-Natural Wines",
    "Natural Acidic", "Natural Sweet", "Natural Tannic",
    "Non-Natural Acidic", "Non-Natural Sweet", "Non-Natural Tannic"
))

# Print each row
for row in results:
    country_name, natural_wines_count, non_natural_wines_count, \
    natural_acidic_wines_count, natural_sweet_wines_count, natural_tannic_wines_count, \
    non_natural_acidic_wines_count, non_natural_sweet_wines_count, non_natural_tannic_wines_count = row
    print("{:<20} {:<20} {:<20} {:<20} {:<20} {:<20} {:<20} {:<20} {:<20}".format(
        country_name, natural_wines_count, non_natural_wines_count,
        natural_acidic_wines_count, natural_sweet_wines_count, natural_tannic_wines_count,
        non_natural_acidic_wines_count, non_natural_sweet_wines_count, non_natural_tannic_wines_count
    ))

Country              Natural Wines        Non-Natural Wines    Natural Acidic       Natural Sweet        Natural Tannic       Non-Natural Acidic   Non-Natural Sweet    Non-Natural Tannic  
Italie               18                   365                  12                   1                    2                    201                  67                   62                  
France               13                   318                  10                   0                    0                    121                  0                    53                  
Espagne              2                    92                   2                    0                    0                    39                   0                    34                  
États-Unis           0                    74                   0                    0                    0                    41                   0                    26                  
Portugal             0                    28           

In [59]:
query2 = """
    SELECT wines.name
    FROM wines
    JOIN regions ON wines.region_id = regions.id
    JOIN countries ON regions.country_code = countries.code
    JOIN most_used_grapes_per_country ON countries.code = most_used_grapes_per_country.country_code
    JOIN grapes ON most_used_grapes_per_country.grape_id = grapes.id
    WHERE grapes.name = 'Cabernet Sauvignon'
    GROUP BY wines.name
    ORDER BY 
        ((wines.ratings_average * wines.ratings_count) + 
        ((
            SELECT SUM(wines.ratings_average) 
            FROM wines
        ) / (
            SELECT COUNT(DISTINCT id) 
            FROM wines
        ))
    ) / (wines.ratings_count + 1) DESC
    LIMIT 5;
"""
query3 = """
    SELECT wines.name, ((wines.ratings_average * wines.ratings_count) + (SUM(wines.ratings_average) / COUNT( DISTINCT wines.id))) / (wines.ratings_count + 1) AS weighted_rating
    FROM wines
    """
cursor.execute(query2)
results = cursor.fetchall()
print(results)

[('Mágico',), ('Unico',), ('Special Selection Cabernet Sauvignon',), ('Grange',), ('Unico Reserva Especial Edición',)]
